# PsychChip SNP Annotation Workflow

#### Last Updated 15-Sep-2016

In [1]:
import sys
import os
import subprocess
import numpy as np
import pandas as pd
import bcolz as bc

In [2]:
os.chdir('/Users/mooneymi/Documents/ADHD/Genotypes')

## Load Array Manifest

In [3]:
## Load PsychChip Manifest
lumi_manifest = pd.read_csv('Illumina_PsychChip_v1-1_15073391.csv', header=7, index_col=1, 
                            dtype={'Name':'str','Chr':'str','MapInfo':'str'})

In [4]:
## Remove controls
lumi_manifest = lumi_manifest[0:603132]

In [5]:
## Convert column types
lumi_manifest['Chr'] = lumi_manifest['Chr'].astype('str')
lumi_manifest['MapInfo'] = lumi_manifest['MapInfo'].astype(int)

## Extract SNP Names

This simply involves removing prefixes (e.g. psy_, exm-, etc.) from Rs IDs.

In [6]:
## Clean SNP names
lumi_manifest['Name_cleaned'] = pd.Series(lumi_manifest.index, index=lumi_manifest.index)
lumi_manifest['Name_cleaned'] = lumi_manifest['Name_cleaned'].str.replace('psy_rs', 'rs')
lumi_manifest['Name_cleaned'] = lumi_manifest['Name_cleaned'].str.replace('exm-rs', 'rs')
lumi_manifest['Name_cleaned'] = lumi_manifest['Name_cleaned'].str.replace('newrs', 'rs')
lumi_manifest['Name_cleaned'] = lumi_manifest['Name_cleaned'].str.replace('snv-rs', 'rs')

## Identify Duplicate SNPs

SNPs can be duplicated either by name (now that prefixes have been removed) or by position.

In [7]:
## Flag duplicates by postion
lumi_manifest['Position'] = lumi_manifest['Chr'] + ':' + lumi_manifest['MapInfo'].astype('str')
lumi_manifest['Position_dup'] = lumi_manifest['Position'].duplicated(keep=False)
sum(lumi_manifest['Position_dup'])

18552

In [9]:
## Flag duplicates by Name
lumi_manifest['Name_cleaned_dup'] = lumi_manifest['Name_cleaned'].duplicated(keep=False)
sum(lumi_manifest['Name_cleaned_dup'])

2848

In [10]:
## Write position duplicates to file
lumi_manifest.loc[lumi_manifest.Position_dup].to_csv('Illumina_PsychChip_v1-1_15073391_dup_positions.csv')

In [11]:
## Write name duplicates to file
lumi_manifest.loc[lumi_manifest.Name_cleaned_dup].to_csv('Illumina_PsychChip_v1-1_15073391_dup_names.csv')

## Assign Positions for Unmapped SNPs

Use dbSNP to add positions for SNPs that match by name. Currently this is only done for SNPs without a position in the manifest, but it could be done for all SNPs to check locations (this would be done again with SHAPEIT).

In [65]:
## Print number of SNPs that do not have a postion in the manifest
sum((lumi_manifest.Chr=='0'))

1964

In [39]:
## Assign dbSNP positions for unmapped SNPs
lumi_manifest['Chr_dbSNP'] = pd.Series(np.nan, index=lumi_manifest.index)
lumi_manifest['Chr_dbSNP'] = lumi_manifest['Chr_dbSNP'].astype('str')
lumi_manifest['MapInfo_dbSNP'] = pd.Series(np.nan, index=lumi_manifest.index)

chroms = [str(i) for i in range(1,23)]
chroms.append('X')

lumi_manifest_chr = lumi_manifest.loc[(lumi_manifest.Chr == '0'),]

## Get array of unique IDs on chromosome
chr_names = lumi_manifest_chr['Name_cleaned'].unique()
#len(chr_names)

## Load dbSNP into ctables
## Create empty bcolz ctable
dbsnp_chr_sub = bc.zeros(0, dtype=zip(('Name','Chr','Position'),('S20','S4','int64')))

## Read data into pandas dataframe in chunks and append to ctable
for c in chroms:
    dbsnp_file = './dbSNP_b147_GRCh37p13/chr_' + c + '.txt'
    chunks = pd.read_table(dbsnp_file, header=None, skiprows=7, 
                           usecols=[0, 6, 11], names=['Name','Chr','Position'], 
                           dtype={'Name':'str','Chr':'str','Position':'str'}, na_values=[' ', ''], chunksize=1000000)
    for chunk in chunks:
        chunk['Name'] = 'rs' + chunk['Name']
        chunk_sub = chunk.loc[(chunk['Name'].isin(chr_names)) & (pd.notnull(chunk['Position'])),]
        if chunk_sub.shape[0] > 0:
            dbsnp_chr_sub.append(bc.ctable.fromdataframe(chunk_sub))

## Print size of bcolz table (number of dbSNPs that match unmapped SNP names)
print dbsnp_chr_sub.size
    
## Map dbSNP ID based on position
for row in lumi_manifest.loc[lumi_manifest.Chr == '0',].itertuples():
    idx = row.Index
    rs_id = str(row.Name_cleaned)
    pos = [(dbsnp.Chr, dbsnp.Position) for dbsnp in dbsnp_chr_sub.where("(Name == rs_id)")]
    if len(pos) == 1:
        lumi_manifest.set_value(idx, 'Chr_dbSNP', pos[0][0])
        lumi_manifest.set_value(idx, 'MapInfo_dbSNP', pos[0][1])


1114


In [41]:
## Replace missing values in Chr_dbSNP
lumi_manifest.Chr_dbSNP.replace('nan', np.nan, inplace=True)

In [64]:
## Print number of SNPs that still do not have a position
sum((lumi_manifest.Chr=='0') & (pd.isnull(lumi_manifest.Chr_dbSNP)))

850

## Assign Rs IDs from dbSNP

For any SNPs in the manifest that have a location that matches a dbSNP SNP, the corresponding Rs ID is assigned.

In [47]:
## Find dbSNP IDs based on position
lumi_manifest['dbSNP_RsID'] = pd.Series(np.nan, index=lumi_manifest.index)
lumi_manifest['dbSNP_RsID'] = lumi_manifest['dbSNP_RsID'].astype('str')
lumi_manifest['dbSNP_RsID_count'] = pd.Series(np.nan, index=lumi_manifest.index)

chroms = [str(i) for i in range(1,23)]
chroms.append('X')

for c in chroms:
    lumi_manifest_chr = lumi_manifest.loc[(lumi_manifest.Chr == c),]
    ## Get array of unique positions on chromosome
    chr_positions = lumi_manifest_chr['MapInfo'].astype('str').unique()
    #len(chr_positions)
    
    ## Get array of unique IDs on chromosome
    chr_names = lumi_manifest_chr['Name_cleaned'].unique()
    #len(chr_names)
    
    ## Load dbSNP into ctables
    ## Create empty bcolz ctable
    dbsnp_chr_sub = bc.zeros(0, dtype=zip(('Name','Chr','Position'),('S20','S4','int64')))
    
    ## Read data into pandas dataframe in chunks and append to ctable
    dbsnp_file = './dbSNP_b147_GRCh37p13/chr_' + c + '.txt'
    chunks = pd.read_table(dbsnp_file, header=None, skiprows=7, 
                           usecols=[0, 6, 11], names=['Name','Chr','Position'], 
                           dtype={'Name':'str','Chr':'str','Position':'str'}, na_values=[' '], chunksize=1000000)
    for chunk in chunks:
        chunk['Name'] = 'rs' + chunk['Name']
        chunk_sub = chunk.loc[(chunk['Position'].isin(chr_positions)) | (chunk['Name'].isin(chr_names)),]
        dbsnp_chr_sub.append(bc.ctable.fromdataframe(chunk_sub))
    
    print c, dbsnp_chr_sub.size
    
    ## Map dbSNP ID based on position
    for row in lumi_manifest.loc[lumi_manifest.Chr == c,].itertuples():
        idx = row.Index
        chrom = str(row.Chr)
        pos = int(row.MapInfo)
        rs_ids = [dbsnp.Name for dbsnp in dbsnp_chr_sub.where("(Chr == chrom) & (Position == pos)")]
        num_rs = len(rs_ids)
        if num_rs == 0:
            lumi_manifest.set_value(idx, 'dbSNP_RsID_count', 0)
        else:
            rs_ids = ','.join(rs_ids)
            lumi_manifest.set_value(idx, 'dbSNP_RsID', rs_ids)
            lumi_manifest.set_value(idx, 'dbSNP_RsID_count', num_rs)


1 53022
2 46008
3 38691
4 31797
5 32096
6 52384
7 30533
8 26941
9 25042
10 26746
11 33163
12 29521
13 16610
14 18862
15 18746
16 21713
17 23933
18 13530
19 24651
20 14684
21 7748
22 10446
X 15921


In [50]:
## Replace missing values in dbSNP_RsID
lumi_manifest.dbSNP_RsID.replace('nan', np.nan, inplace=True)

In [51]:
## Print the number of SNPs that do not have an assigned Rs ID from dbSNP
sum(pd.isnull(lumi_manifest['dbSNP_RsID']))

15004

In [52]:
## Check whether an RsID is available for each SNP
lumi_manifest['RsID_available'] = lumi_manifest.Name_cleaned.str.contains(r'^rs[0-9]+') | pd.notnull(lumi_manifest['dbSNP_RsID'])

In [62]:
## Print the number of SNPs that do not have an Rs ID (originally in manifest or assigned from dbSNP)
sum(np.logical_not(lumi_manifest['RsID_available']))

12170

** Clearly there are some SNP IDs that are no longer in the current dbSNP database

In [56]:
lumi_manifest.loc[(lumi_manifest.RsID_available) & (pd.isnull(lumi_manifest.dbSNP_RsID))].head(25)

,IlmnID,IlmnStrand,SNP,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,GenomeBuild,Chr,MapInfo,...,RefStrand,Name_cleaned,Position,Position_dup,Name_cleaned_dup,dbSNP_RsID,dbSNP_RsID_count,Chr_dbSNP,MapInfo_dbSNP,RsID_available
Name,,,,,,,,,,,,,,,,,,,,,
exm-rs10862691,exm-rs10862691-131_B_F_1990478207,BOT,[T/C],39802919.0,TCTGTAGATGTCTATTAGGTCTGCTTGTTGCAGAGCTGAGTTCAGG...,NaN,NaN,37.0,0,0,...,+,rs10862691,0:0,True,False,NaN,NaN,12,83941693.0,True
exm-rs11136341,exm-rs11136341-131_T_F_1990487521,TOP,[A/G],13798921.0,CCCCATCCAGGGCAGATTCCTCTGGGGCTGAGGCTACTGCAGCCTG...,NaN,NaN,37.0,0,0,...,+,rs11136341,0:0,True,False,NaN,NaN,8,145043543.0,True
exm-rs1236440,exm-rs1236440-131_T_F_1990488040,TOP,[A/G],39658135.0,GGAGAAACATTCCACGCTTGAGGATAAGAAAAATGAGTATTGTGAA...,NaN,NaN,37.0,Y,15333149,...,-,rs1236440,Y:15333149,False,False,NaN,NaN,NaN,NaN,True
exm-rs13201313,exm-rs13201313-131_T_R_1990488446,TOP,[A/C],81619854.0,TCTGCGTCTTAGGGCTTGGACATATGTGTGAGGCCATTATTTTGGA...,NaN,NaN,37.0,6,29813829,...,-,rs13201313,6:29813829,False,False,NaN,0.0,NaN,NaN,True
exm-rs13303755,exm-rs13303755-131_T_R_1990488488,TOP,[A/C],50746189.0,CAAGACATGTCTCAAAAGGGTATTTAGGATTCTCATGCCAGGAAAG...,NaN,NaN,37.0,Y,23612197,...,-,rs13303755,Y:23612197,False,False,NaN,NaN,NaN,NaN,True
exm-rs13304625,exm-rs13304625-131_B_R_1990479411,BOT,[T/C],62782235.0,CAGCAGCAGTAGATTCTCATATGAGCAATAACTCCATAATGAACTG...,NaN,NaN,37.0,Y,19077409,...,-,rs13304625,Y:19077409,False,False,NaN,NaN,NaN,NaN,True
exm-rs13447352,exm-rs13447352-131_T_F_1990488553,TOP,[A/C],40736245.0,ACAGTATGTGGGATTTTTTTAGATGTGTTCAATTTGAAAGTAACTT...,NaN,NaN,37.0,Y,22749853,...,+,rs13447352,Y:22749853,False,False,NaN,NaN,NaN,NaN,True
exm-rs13447360,exm-rs13447360-131_P_F_1990486710,PLUS,[D/I],81634283.0,TTTCAGTTACTTAGATGGTCTCATAAGGTTTCTGATACAATTTGAA...,NaN,NaN,37.0,Y,22754671,...,+,rs13447360,Y:22754671,False,False,NaN,NaN,NaN,NaN,True
exm-rs13447378,exm-rs13447378-131_B_F_1990479458,BOT,[G/C],56742203.0,TGTGTTTCCATTTCTCTTTTCCTCATTTCTCATCATCTACATTTCT...,15605269.0,TGTGTTTCCATTTCTCTTTTCCTCATTTCTCATCATCTACATTTCT...,37.0,Y,22741740,...,+,rs13447378,Y:22741740,False,False,NaN,NaN,NaN,NaN,True


## Save Updated Manifest to File

In [57]:
lumi_manifest.to_csv('Illumina_PsychChip_v1-1_15073391_editedMM.csv')

## Check that Manifest and dbSNP Positions Match

In [ ]:
## Check that manifest and dbSNP positions match
lumi_manifest['dbSNP_position_mismatch'] = pd.Series(np.nan, index=lumi_manifest.index)